In [1]:
"""
Step4
當用戶發貼圖，就回貼圖(亂數抽取)

目標:
學會接收StickerMessage事件

"""

'\nStep4\n當用戶發貼圖，就回貼圖(亂數抽取)\n\n目標:\n學會接收StickerMessage事件\n\n'

In [2]:
# 區塊1
# 匯入套件

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, FollowEvent, ImageSendMessage, StickerSendMessage, StickerMessage
)

import json
import random

In [3]:
# 區塊2
# 啟用API
# 要先從 secretFile.txt 取出 channelAccessToken 和 channelSecret
# 多增加 linkRichmenuToUser 方法
# 多增加 getProfile 方法

app = Flask(__name__)

secretFile = json.load(open("../secretFile.txt","r"))
richmenuId = secretFile['richmenuId']
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']
userId = secretFile['userId']

line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)

def linkRichmenuToUser():
    line_bot_api.link_rich_menu_to_user(userId, richmenuId)
    
def getProfile():

    profile = line_bot_api.get_profile(userId)
    
    print(profile.display_name)
    print(profile.user_id)
    print(profile.picture_url)
    print(profile.status_message)
    
    return profile

In [4]:
# 區塊3
# 設置入口

# 從 Line 的 Webhook URL 接到這裡
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

In [5]:
# 區塊4-1
# 設定導流的事件
# 收到 FollowEvent

@handler.add(FollowEvent)
def Follow_message(event):
    
    # 拿到 getProfile 物件，
    profile = getProfile()
    
    # 取得 display_name屬性
    displayName = profile.display_name
    
    # 拿到 getProfile 物件，取得 picture_url屬性
    pictureUrl = profile.picture_url
    
    # 放入等一下要傳的訊息
    sendMessage = [
        
        # 文字消息1
        TextSendMessage(
            text="哈囉%s，打開下面選單可以看到我們提供的服務喔!" % (displayName)
        ),
        
        # 文字消息2
        TextSendMessage(
            text="喔對了，這是你醜醜的照片"
        ),
        
        # 圖片消息
        ImageSendMessage(
            original_content_url= pictureUrl,
            preview_image_url= pictureUrl
        ),
        
        # 貼圖消息
        StickerSendMessage(
            package_id='11537',
            sticker_id='51626512'
        )
        
    ]
    
    # 發送歡迎詞
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage)
    
    # 綁定richmenu
    linkRichmenuToUser()

In [6]:
# 區塊4-2
# 設定導流的事件
# 收到 MessageEvent
# MessageEvent子分類-StickerMessage


@handler.add(MessageEvent,message = StickerMessage)
def stickerMessage(event):
    
    # 隨機抽取貼圖編號
    stickerId = str(random.randint(51626494,51626533))
    
    
    sendMessage = [
        StickerSendMessage(
            package_id='11537',
            sticker_id=stickerId
        )
    ]
    
    line_bot_api.reply_message(
        event.reply_token,
        sendMessage
    )

In [ ]:
# 區塊5
# 啟動Server

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [30/Apr/2019 05:06:23] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [30/Apr/2019 05:06:28] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [30/Apr/2019 05:06:31] "POST / HTTP/1.1" 200 -
